In [ ]:
import requests
import pandas as pd

# Correct API Endpoint for Treasury Securities Auctions Data
API_ENDPOINT = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query"

# Parameters for filtering (modify as needed)
params = {
    "fields": "record_date, cusip, security_type, security_term, auction_date, price_per100,maturity_date,allocation_pctage,bid_to_cover_ratio",  # Specify fields
    "sort": "-auction_date",  # Sort by auction_date descending (newest first)
    "page[number]": 1,  # Pagination - page number
    "page[size]": 200  # Number of records per page (last 10 auctions)
}

# Make the API request
response = requests.get(API_ENDPOINT, params=params)

if response.status_code == 200:
    data = response.json()
    # Extract auction records
    records = data.get("data", [])
    # Convert to DataFrame for better readability
    df = pd.DataFrame(records)
    print(df)  # Display the last 10 auctions
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    print(response.text)  # Print the error message


    record_date      cusip security_type     security_term auction_date  \
0    2024-12-31  91282CMC2          Note            7-Year   2024-12-26   
1    2024-12-27  91282CLT6      FRN Note   1-Year 10-Month   2024-12-24   
2    2024-12-31  91282CMD0          Note            5-Year   2024-12-24   
3    2024-12-31  91282CME8          Note            2-Year   2024-12-23   
4    2024-12-26  912797NU7          Bill           52-Week   2024-12-23   
..          ...        ...           ...               ...          ...   
195  2024-07-23  912797LH8          Bill            8-Week   2024-07-18   
196  2024-07-23  912797KY2          Bill            4-Week   2024-07-18   
197  2024-07-31  912810UB2          Bond  19-Year 10-Month   2024-07-17   
198  2024-07-23  912797MC8          Bill           17-Week   2024-07-17   
199  2024-07-18  912797KD8           CMB            42-Day   2024-07-16   

    price_per100 maturity_date allocation_pctage bid_to_cover_ratio  
0           null    2031-12-3

In [ ]:
df.index = df['auction_date']
df

,record_date,cusip,security_type,security_term,auction_date,price_per100,maturity_date,allocation_pctage,bid_to_cover_ratio
auction_date,,,,,,,,,
2024-12-26,2024-12-31,91282CMC2,Note,7-Year,2024-12-26,null,2031-12-31,null,null
2024-12-24,2024-12-27,91282CLT6,FRN Note,1-Year 10-Month,2024-12-24,null,2026-10-31,null,null
2024-12-24,2024-12-31,91282CMD0,Note,5-Year,2024-12-24,null,2029-12-31,null,null
2024-12-23,2024-12-31,91282CME8,Note,2-Year,2024-12-23,null,2026-12-31,null,null
2024-12-23,2024-12-26,912797NU7,Bill,52-Week,2024-12-23,null,2025-12-26,null,null
...,...,...,...,...,...,...,...,...,...
2024-07-18,2024-07-23,912797LH8,Bill,8-Week,2024-07-18,99.181778,2024-09-17,93.230000,2.680000
2024-07-18,2024-07-23,912797KY2,Bill,4-Week,2024-07-18,99.590111,2024-08-20,68.260000,2.590000
2024-07-17,2024-07-31,912810UB2,Bond,19-Year 10-Month,2024-07-17,102.062500,2044-05-15,92.560000,2.680000


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive


from google.colab import output
output.enable_custom_widget_manager()


# Ensure datetime and numeric types
df['auction_date'] = pd.to_datetime(df['auction_date'])
df['maturity_date'] = pd.to_datetime(df['maturity_date'])
df['bid_to_cover_ratio'] = pd.to_numeric(df['bid_to_cover_ratio'], errors='coerce')

# Create a new column for the week of the auction date (using the start of the week)
df['auction_week'] = df['auction_date'].dt.to_period('W')

# Sort by maturity_date to get the correct order for the bar plot
df = df.sort_values(by='maturity_date')

# Get the unique weeks
weeks = df['auction_week'].unique()

# Function to update the plot based on the selected week
def update_plot(selected_week):
    # Filter data for the selected week
    week_data = df[df['auction_week'] == selected_week]

    # Create the bar plot
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=week_data['maturity_date'],
        y=week_data['bid_to_cover_ratio'],
        text=week_data['security_type'],
        marker=dict(color=week_data['bid_to_cover_ratio'], colorscale='Viridis'),
        hoverinfo='text+x+y',
    ))

    # Update layout
    fig.update_layout(
        title=f'Auction Bid-to-Cover Ratio vs Maturity Date for Week: {selected_week}',
        xaxis_title='Maturity Date',
        yaxis_title='Bid-to-Cover Ratio',
        xaxis_tickangle=45,
        barmode='group',
        plot_bgcolor='white',
        # template='plotly_dark',
        showlegend=False
    )

    # Show plot
    fig.show()

# Create an interactive widget for selecting the week
interactive_plot = interactive(update_plot, selected_week=widgets.Dropdown(
    options=[(str(week), week) for week in weeks],
    value=weeks[0],  # Default to the first week
    description='Select Week:',
    style={'description_width': 'initial'}
))

# Display the interactive plot
interactive_plot


interactive(children=(Dropdown(description='Select Week:', options=(('2024-07-15/2024-07-21', Period('2024-07-…

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Remove rows where 'bid_to_cover_ratio' is NaN
df = df.dropna(subset=['bid_to_cover_ratio'])
df['auction_date'] = pd.to_datetime(df['auction_date'])
df['maturity_date'] = pd.to_datetime(df['maturity_date'])
df['bid_to_cover_ratio'] = pd.to_numeric(df['bid_to_cover_ratio'], errors='coerce')

# Create a new column for the week of the auction date (using the start of the week)
df['auction_week'] = df['auction_date'].dt.to_period('W')

# Sort by maturity_date to get the correct order for the line plot
df = df.sort_values(by='maturity_date')

# Get the unique weeks
weeks = df['auction_week'].unique()


# Select only the last 12 weeks
weeks = weeks[-12:]

# Convert periods to timestamps (start of the week)
week_start_dates = [week.start_time for week in weeks]

# Create an empty figure for the plot
fig = go.Figure()

# Define a function to get the opacity based on the age of the auction week
def get_opacity(auction_week, max_weeks):
    # Convert auction week and max_weeks to start dates
    auction_week_start = auction_week.start_time
    max_weeks_start = max_weeks.start_time

    # Calculate the difference in days between the current week and the most recent week
    diff_days = np.abs(np.datetime64(auction_week_start) - np.datetime64(max_weeks_start)) / np.timedelta64(1, 'D')

    # Apply an exponential decay for opacity
    decay_factor = np.exp(-diff_days / 10)
    opacity = max(0.1, decay_factor)

    return opacity

# Plot a line for each auction week, with adjusted opacity and scatter points
for i, week in enumerate(weeks):
    week_data = df[df['auction_week'] == week]


    # For the most recent auction week (last in the list), set color to red
    if i == len(weeks) - 1:
        color = 'rgba(255, 0, 0, 1)'
    else:
        opacity = get_opacity(week, weeks[-1])
        color = f'rgba(0, 100, 255, {opacity})'

    # Add the line (term structure) for the current week
    fig.add_trace(go.Scatter(
        x=week_data['maturity_date'],
        y=week_data['bid_to_cover_ratio'],
        mode='lines',
        line=dict(width=4, color=color),
        name=str(week),
        showlegend=True
    ))

    # Add the scatter points with security_type in hover text
    fig.add_trace(go.Scatter(
        x=week_data['maturity_date'],
        y=week_data['bid_to_cover_ratio'],
        mode='markers',
        marker=dict(size=8, color=color, line=dict(width=2, color=color)),  # Points with color
        name=str(week),
        text=week_data['security_type'],
        hoverinfo='text',
        showlegend=False
    ))

# Update layout of the plot
fig.update_layout(
    title='Auction Bid-to-Cover Ratio per Maturity (Last 12 Weeks)',
    xaxis_title='Maturity Date',
    yaxis_title='Bid-to-Cover Ratio',
    xaxis_tickangle=45,
    plot_bgcolor='white',
    # template='plotly_dark',
    showlegend=True,
    title_x=0.5
)

# Show the figure
fig.show()


,record_date,cusip,security_type,security_term,auction_date,price_per100,maturity_date,allocation_pctage,bid_to_cover_ratio,auction_week
auction_date,,,,,,,,,,
2024-07-16,2024-07-18,912797KD8,CMB,42-Day,2024-07-16,99.385750,2024-08-29,29.210000,2.92,2024-07-15/2024-07-21
2024-07-17,2024-07-23,912797MC8,Bill,17-Week,2024-07-17,98.302597,2024-11-19,82.770000,3.12,2024-07-15/2024-07-21
2024-07-17,2024-07-31,912810UB2,Bond,19-Year 10-Month,2024-07-17,102.062500,2044-05-15,92.560000,2.68,2024-07-15/2024-07-21
2024-07-18,2024-07-23,912797KY2,Bill,4-Week,2024-07-18,99.590111,2024-08-20,68.260000,2.59,2024-07-15/2024-07-21
2024-07-18,2024-07-31,91282CLE9,TIPS Note,10-Year,2024-07-18,100.012919,2034-07-15,69.870000,2.38,2024-07-15/2024-07-21
...,...,...,...,...,...,...,...,...,...,...
2024-12-17,2024-12-31,912810UF3,Bond,19-Year 11-Month,2024-12-17,99.206641,2044-11-15,81.640000,2.50,2024-12-16/2024-12-22
2024-12-18,2024-12-24,912797PA9,Bill,17-Week,2024-12-18,98.605056,2025-04-22,75.520000,2.97,2024-12-16/2024-12-22
2024-12-19,2024-12-24,912797MY0,Bill,4-Week,2024-12-19,99.671000,2025-01-21,11.980000,2.93,2024-12-16/2024-12-22


In [ ]:
week_data.sort_index()


,record_date,cusip,security_type,security_term,auction_date,price_per100,maturity_date,allocation_pctage,bid_to_cover_ratio,auction_week
auction_date,,,,,,,,,,
2024-12-16,2024-12-19,912797KJ5,Bill,13-Week,2024-12-16,98.925694,2025-03-20,15.160000,2.62,2024-12-16/2024-12-22
2024-12-16,2024-12-19,912797NV5,Bill,26-Week,2024-12-16,97.885333,2025-06-20,65.040000,3.29,2024-12-16/2024-12-22
2024-12-17,2024-12-19,912797LZ8,CMB,42-Day,2024-12-17,99.501833,2025-01-30,90.240000,2.59,2024-12-16/2024-12-22
2024-12-17,2024-12-31,912810UF3,Bond,19-Year 11-Month,2024-12-17,99.206641,2044-11-15,81.640000,2.50,2024-12-16/2024-12-22
2024-12-18,2024-12-24,912797PA9,Bill,17-Week,2024-12-18,98.605056,2025-04-22,75.520000,2.97,2024-12-16/2024-12-22
2024-12-19,2024-12-24,912797MY0,Bill,4-Week,2024-12-19,99.671000,2025-01-21,11.980000,2.93,2024-12-16/2024-12-22
2024-12-19,2024-12-24,912797NH6,Bill,8-Week,2024-12-19,99.342000,2025-02-18,35.230000,2.98,2024-12-16/2024-12-22
2024-12-19,2024-12-31,91282CLV1,TIPS Note,4-Year 10-Month,2024-12-19,98.060344,2029-10-15,15.010000,2.10,2024-12-16/2024-12-22
